In [1]:
# Import PySpark
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler


In [2]:
# create a SparkSession object
spark = SparkSession.builder.appName('ImportCSV').getOrCreate()

# read the CSV file into a DataFrame
df = spark.read.options(header=True, inferSchema=True).csv('Data/creditcard.csv')

# show the DataFrame
df.show()

23/04/17 11:27:27 WARN Utils: Your hostname, server-gpu resolves to a loopback address: 127.0.1.1; using 192.168.99.222 instead (on interface enp9s0)
23/04/17 11:27:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/17 11:27:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/17 11:27:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|                V3|                 V4|                 V5|                 V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|                V14|                V15|                V16|                 V17|                V18|                V19|                V20|                 V

In [3]:
df.head(5)

[Row(Time=0.0, V1=-1.3598071336738, V2=-0.0727811733098497, V3=2.53634673796914, V4=1.37815522427443, V5=-0.338320769942518, V6=0.462387777762292, V7=0.239598554061257, V8=0.0986979012610507, V9=0.363786969611213, V10=0.0907941719789316, V11=-0.551599533260813, V12=-0.617800855762348, V13=-0.991389847235408, V14=-0.311169353699879, V15=1.46817697209427, V16=-0.470400525259478, V17=0.207971241929242, V18=0.0257905801985591, V19=0.403992960255733, V20=0.251412098239705, V21=-0.018306777944153, V22=0.277837575558899, V23=-0.110473910188767, V24=0.0669280749146731, V25=0.128539358273528, V26=-0.189114843888824, V27=0.133558376740387, V28=-0.0210530534538215, Amount=149.62, Class=0),
 Row(Time=0.0, V1=1.19185711131486, V2=0.26615071205963, V3=0.16648011335321, V4=0.448154078460911, V5=0.0600176492822243, V6=-0.0823608088155687, V7=-0.0788029833323113, V8=0.0851016549148104, V9=-0.255425128109186, V10=-0.166974414004614, V11=1.61272666105479, V12=1.06523531137287, V13=0.48909501589608, V14=-

In [4]:
# group the DataFrame by all columns
grouped_df = df.groupBy(df.columns).count()

# select rows where count is greater than 1
duplicates = grouped_df.filter('count > 1')

In [5]:
duplicates.show(), duplicates.count()

+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------+-----+-----+
|   Time|                V1|                V2|                V3|                 V4|                 V5|                 V6|                 V7|                V8|                V9|               V10|                V11|               V12|                V13|                 V14|                V15|                V16|                V17|               V18|               V19|                V20|                V

(None, 773)

In [6]:
# remove duplicates
df = df.dropDuplicates()

In [7]:
# group the DataFrame by all columns
grouped_df = df.groupBy(df.columns).count()

# select rows where count is greater than 1
duplicates = grouped_df.filter('count > 1')

In [8]:
duplicates.show(), duplicates.count()


+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+-----+
|Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|count|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+-----+
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+-----+



(None, 0)

In [9]:
# select the feature columns and the label column
label = df.select('Class')
feature = df.drop('Class')


In [10]:
feature.head(3), feature.columns

([Row(Time=26.0, V1=-0.529912284186556, V2=0.873891581460326, V3=1.34724732930113, V4=0.145456676582257, V5=0.414208858362661, V6=0.10022309405219, V7=0.711206082959649, V8=0.1760659570625, V9=-0.286716934699997, V10=-0.484687683196852, V11=0.872489590125871, V12=0.851635859904339, V13=-0.571745302934562, V14=0.100974273045751, V15=-1.51977183258512, V16=-0.284375978261788, V17=-0.310523584869201, V18=-0.404247868800905, V19=-0.823373523914155, V20=-0.290347610865436, V21=0.0469490671140629, V22=0.208104855076299, V23=-0.185548346773547, V24=0.00103065983293288, V25=0.0988157011025622, V26=-0.552903603040518, V27=-0.0732880835681738, V28=0.0233070451077205, Amount=6.14),
  Row(Time=158.0, V1=-0.600816388115364, V2=0.922454525911535, V3=-0.135951820418704, V4=-1.25991483910974, V5=2.43982440634166, V6=3.33020532398973, V7=0.0871070880575762, V8=0.949659242063477, V9=-0.327931944022326, V10=0.0617085198149349, V11=-0.157387072529051, V12=-0.233220336669169, V13=-0.0533324475993436, V14=0

In [11]:
label.head(10), label.columns

([Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0)],
 ['Class'])

I don't find a way to easily oversmapling and undersampling so use pandas

In [12]:
p_feature = feature.toPandas()
p_label = label.toPandas()

In [13]:
p_feature.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,26.0,-0.529912,0.873892,1.347247,0.145457,0.414209,0.100223,0.711206,0.176066,-0.286717,...,-0.290348,0.046949,0.208105,-0.185548,0.001031,0.098816,-0.552904,-0.073288,0.023307,6.14
1,158.0,-0.600816,0.922455,-0.135952,-1.259915,2.439824,3.330205,0.087107,0.949659,-0.327932,...,0.348028,-0.311489,-0.927233,-0.045268,0.979587,0.016423,0.091043,0.371280,0.180187,1.79
2,190.0,-1.505779,-0.215325,1.991294,-1.631493,-0.635965,0.228414,-0.034266,0.042696,-0.235440,...,0.232752,-0.101938,0.148146,-0.450115,-0.396558,0.554224,-0.340493,-0.335618,-0.413379,82.29
3,265.0,-0.491003,0.906953,1.645423,-0.083531,-0.195560,-0.710165,0.559119,0.116340,-0.538190,...,0.062757,-0.168067,-0.517387,0.018650,0.491652,-0.277795,0.043841,0.253372,0.111749,9.03
4,459.0,-0.528218,0.981232,1.652988,-0.150715,0.137048,-0.203980,0.583370,0.165246,-0.781696,...,0.127768,-0.123569,-0.272731,0.023887,0.223936,-0.297777,0.068300,0.281018,0.111557,6.99


In [14]:
p_label.head()

,Class
0,0
1,0
2,0
3,0
4,0


NearMiss and SMOTE

In [15]:
from imblearn.under_sampling import NearMiss
# Create an instance of NearMiss
nm = NearMiss()

# Fit and apply NearMiss to downsample the majority class
nm_features, nm_labels = nm.fit_resample(p_feature, p_label)

nm_features.shape, nm_labels.shape, nm_labels.value_counts()

((946, 30),
 (946, 1),
 Class
 0        473
 1        473
 dtype: int64)

In [16]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.metrics import classification_report_imbalanced

smote=SMOTE()
sm_features,sm_labels=smote.fit_resample(p_feature,p_label)
sm_features.shape,sm_labels.shape, sm_labels.value_counts()

((566506, 30),
 (566506, 1),
 Class
 0        283253
 1        283253
 dtype: int64)

Rejoin with feature and label

In [18]:
import pandas as pd

In [23]:
sm_dataset = pd.concat([sm_fea, labels_df], axis=1)
sm_dataset.head()

AttributeError: module 'pandas' has no attribute 'join'

train test split for nm

In [ ]:
from sklearn.model_selection import train_test_split
# split the data into training and test sets
nm_features_train, nm_features_test, nm_label_train, nm_label_test = train_test_split(nm_features, nm_labels, test_size=0.2, random_state=42)

In [ ]:
nm_features_train.shape, nm_label_train.shape, nm_features_test.shape, nm_label_test.shape

import back to pyspark

In [ ]:
nm_features_train.head(3),nm_label_train.head(3), nm_features_test.head(3), nm_label_test.head(3)

In [ ]:
nm_features.count(), nm_labels.count()

In [ ]:
sm_features = spark.createDataFrame(sm_features)
sm_labels = spark.createDataFrame(sm_labels)

In [ ]:
sm_features.head(3), sm_labels.head(3)

In [ ]:
sm_features.count(), sm_labels.count()

Scale the dataset

In [ ]:
nm_cols_to_scale = nm_features.columns
sm_cols_to_scale = sm_features.columns

Near Miss scale

In [ ]:
# # select the columns to be scaled
# cols_to_scale = ['feature1', 'feature2', 'feature3']

# assemble the selected columns into a vector
assembler = VectorAssembler(inputCols=nm_cols_to_scale, outputCol='features')
assembled_df = assembler.transform(nm_features)

# standardize the features
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withMean=True, withStd=True)
nm_scaled_features_df = scaler.fit(assembled_df).transform(assembled_df)

In [ ]:
nm_scaled_features_df.head(5), nm_scaled_features_df.count()

SMOTE scale

In [ ]:
# # select the columns to be scaled
# cols_to_scale = ['feature1', 'feature2', 'feature3']

# assemble the selected columns into a vector
assembler = VectorAssembler(inputCols=sm_cols_to_scale, outputCol='features')
assembled_df = assembler.transform(sm_features)

# standardize the features
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withMean=True, withStd=True)
sm_scaled_features_df = scaler.fit(assembled_df).transform(assembled_df)

In [ ]:
sm_scaled_features_df.head(5), sm_scaled_features_df.count()

other way

In [ ]:
feature_cols = df.drop('Class').columns
label_col = df.select('Class').columns

In [ ]:
assembler = VectorAssembler(inputCols = feature_cols, outputCol='features')
output = assembler.transform(df)

In [ ]:
output = assembler.transform(df)

In [ ]:
finalised_data = output.select('features', 'Class')
finalised_data.show()

In [ ]:
# split the data into training and test sets
train_data, test_data = finalised_data.randomSplit([0.8, 0.2], seed=42)

# create a LogisticRegression object
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol='Class',maxIter=10, regParam=0.01)

# fit the model on the training data
model = lr.fit(train_data)

# make predictions on the test data
predictions = model.transform(test_data)



Pyspark logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# create the logistic regression model
lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)

# train the model
lrModel = lr.fit(train_data)

# make predictions on the test set
predictions = lrModel.transform(test_data)

# evaluate the model using F1 score
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
f1_score = evaluator.evaluate(predictions)
print(f'F1 score: {f1_score}')
